In [2]:
# 1번 셀 - 환경 설정
import sys
import os

# 현재 작업 디렉토리 확인 및 출력
current_dir = os.getcwd()
print("Current working directory:", current_dir)

# 프로젝트 루트 디렉토리 설정
project_root = os.path.dirname(current_dir)
print("Project root directory:", project_root)

# Python 경로에 프로젝트 루트 추가
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print("Python path:", sys.path)

# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from src.database import DatabaseManager
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

Current working directory: /Users/y/python/skytrend/notebooks
Project root directory: /Users/y/python/skytrend
Python path: ['/Users/y/python/skytrend', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/y/python/skytrend/venv/lib/python3.11/site-packages']


In [3]:
# 2번 셀 - 데이터 로드
try:
    print("데이터 로드 중...")
    # DatabaseManager 인스턴스 생성
    db_manager = DatabaseManager()
    
    # MongoDB에서 데이터 로드
    weather_2022 = pd.DataFrame(db_manager.get_data('weather_2022'))
    sales_2022 = pd.DataFrame(db_manager.get_data('sales_2022'))
    weather_2023 = pd.DataFrame(db_manager.get_data('weather_2023'))
    
    print(f"날씨 데이터 2022: {len(weather_2022)}행")
    print(f"매출 데이터 2022: {len(sales_2022)}행")
    print(f"날씨 데이터 2023: {len(weather_2023)}행")
    
except Exception as e:
    print(f"데이터 로드 중 오류 발생: {str(e)}")
finally:
    if 'db_manager' in locals() and db_manager.client is not None:
        db_manager.client.close()
        print("데이터베이스 연결이 종료되었습니다.")

데이터 로드 중...
날씨 데이터 2022: 365행
매출 데이터 2022: 83573행
날씨 데이터 2023: 365행
데이터베이스 연결이 종료되었습니다.


In [4]:
# 3번 셀 - 데이터 전처리 함수 정의
def preprocess_weather_data(df):
    """날씨 데이터 전처리"""
    weather_df = df.copy()
    
    # 날짜 형식 변환
    weather_df['날짜'] = pd.to_datetime(weather_df['tm'])
    
    # 필요한 컬럼 선택 및 이름 변경
    columns_mapping = {
        'avgTa': '평균기온',
        'minTa': '최저기온',
        'maxTa': '최고기온',
        'sumRn': '강수량',
        'avgRhm': '평균습도',
        'avgWs': '평균풍속',
        'avgPs': '평균기압'
    }
    
    weather_df = weather_df.rename(columns=columns_mapping)
    selected_columns = ['날짜'] + list(columns_mapping.values())
    weather_df = weather_df[selected_columns]
    
    # 숫자형 변환
    numeric_columns = list(columns_mapping.values())
    for col in numeric_columns:
        weather_df[col] = pd.to_numeric(weather_df[col], errors='coerce')
    
    # 결측치 처리
    weather_df = weather_df.fillna({
        '강수량': 0,
        '평균습도': weather_df['평균습도'].median(),
        '평균풍속': weather_df['평균풍속'].median(),
        '평균기압': weather_df['평균기압'].median(),
        '평균기온': weather_df['평균기온'].median(),
        '최저기온': weather_df['최저기온'].median(),
        '최고기온': weather_df['최고기온'].median()
    })
    
    # 이상치 처리
    for col in numeric_columns:
        Q1 = weather_df[col].quantile(0.25)
        Q3 = weather_df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        weather_df[col] = weather_df[col].clip(lower_bound, upper_bound)
    
    return weather_df

In [5]:
# 3.5번 셀 - 개선된 날씨 데이터 전처리
def create_weather_features(df):
    """날씨 데이터 특성 생성"""
    features_df = df.copy()
    
    # 기본 이동평균 특성
    for col in ['평균기온', '강수량', '평균습도']:
        features_df[f'{col}_7일이동평균'] = features_df[col].rolling(window=7, min_periods=1).mean()
        features_df[f'{col}_30일이동평균'] = features_df[col].rolling(window=30, min_periods=1).mean()
    
    # 날씨 변화 특성
    features_df['기온변화'] = features_df['최고기온'] - features_df['최저기온']
    features_df['전일대비기온변화'] = features_df['평균기온'].diff().fillna(0)
    
    # 추가된 날씨 특성
    features_df['체감온도'] = features_df.apply(
        lambda x: x['평균기온'] - 0.55 * (1 - x['평균습도']/100) * 
                 (x['평균기온'] - 14) if x['평균기온'] >= 14 else x['평균기온'],
        axis=1
    )
    
    # 극한 날씨 플래그
    features_df['폭염일'] = (features_df['최고기온'] >= 33).astype(int)
    features_df['한파일'] = (features_df['최저기온'] <= -12).astype(int)
    features_df['강수일'] = (features_df['강수량'] > 0).astype(int)
    features_df['강풍일'] = (features_df['평균풍속'] > features_df['평균풍속'].quantile(0.9)).astype(int)
    
    # 계절성 특성
    features_df['월'] = features_df['날짜'].dt.month
    features_df['요일'] = features_df['날짜'].dt.dayofweek
    features_df['주말'] = features_df['날짜'].dt.dayofweek.isin([5, 6]).astype(int)
    features_df['공휴일'] = 0  # 공휴일 정보는 별도로 추가 필요
    
    return features_df

In [7]:
# 4번 셀 - 데이터 전처리 적용
print("데이터 컬럼 확인:")
print("\nSales 2022 데이터 컬럼명:")
print(sales_2022.columns.tolist())

print("\n실제 상권명 목록:")
print(sales_2022['상권_코드_명'].unique())

# 상권별 데이터 수와 평균 매출액을 확인
print("\n상권별 통계:")
stats = sales_2022.groupby('상권_코드_명').agg({
   '당월_매출_금액': ['count', 'mean']
}).sort_values(('당월_매출_금액', 'mean'), ascending=False)
print(stats.head(10))

데이터 컬럼 확인:

Sales 2022 데이터 컬럼명:
['기준_년분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드', '서비스_업종_코드_명', '당월_매출_금액', '당월_매출_건수', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액', '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수', '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수', '시간대_건수~06_매출_건수', '시간대_건수~11_매출_건수', '시간대_건수~14_매출_건수', '시간대_건수~17_매출_건수', '시간대_건수~21_매출_건수', '시간대_건수~24_매출_건수', '남성_매출_건수', '여성_매출_건수', '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수', '연령대_40_매출_건수', '연령대_50_매출_건수', '연령대_60_이상_매출_건수']

실제 상권명 목록:
['사가정역' '숭곡초등학교' '인현시장' ... '서강대학교' '다산성곽길' '신림현대종합상가']

상권별 통계:
            당월_매출_금액              
               count          mean
상권_코드

In [9]:
# 4.5번 셀 - 매출 특성 생성 함수 정의
def create_sales_features(df):
    """매출 관련 추가 특성 생성"""
    features_df = df.copy()
    
    # 매출액 스케일 보정
    if '당월_매출_금액' in features_df.columns:
        features_df['당월_매출_금액'] = features_df['당월_매출_금액'].clip(
            lower=features_df['당월_매출_금액'].quantile(0.01),
            upper=features_df['당월_매출_금액'].quantile(0.99)
        )
    
    # 시계열 특성 추가
    features_df['요일'] = pd.to_datetime(features_df['날짜']).dt.dayofweek
    features_df['월'] = pd.to_datetime(features_df['날짜']).dt.month
    features_df['분기'] = pd.to_datetime(features_df['날짜']).dt.quarter
    features_df['주말'] = pd.to_datetime(features_df['날짜']).dt.dayofweek.isin([5, 6]).astype(int)
    
    # 상권별 이동평균 특성
    if '당월_매출_금액' in features_df.columns:
        features_df['7일_이동평균_매출'] = features_df.groupby('상권_코드_명')['당월_매출_금액'].transform(
            lambda x: x.rolling(window=7, min_periods=1).mean())
        features_df['30일_이동평균_매출'] = features_df.groupby('상권_코드_명')['당월_매출_금액'].transform(
            lambda x: x.rolling(window=30, min_periods=1).mean())
        
        # 매출 변화율
        features_df['전일대비_매출_변화'] = features_df.groupby('상권_코드_명')['당월_매출_금액'].transform(
            lambda x: x.pct_change()).fillna(0)
        features_df['7일_매출_변화율'] = features_df.groupby('상권_코드_명')['당월_매출_금액'].transform(
            lambda x: x.pct_change(periods=7)).fillna(0)
        
        # 상권별 평균 대비 매출 비율
        features_df['상권평균대비_비율'] = features_df.groupby('상권_코드_명')['당월_매출_금액'].transform(
            lambda x: x / x.mean())
    
    # 계절성 특성 추가
    features_df['계절'] = features_df['월'].map({
        12: '겨울', 1: '겨울', 2: '겨울',
        3: '봄', 4: '봄', 5: '봄',
        6: '여름', 7: '여름', 8: '여름',
        9: '가을', 10: '가을', 11: '가을'
    })
    
    # 계절별 더미 변수 생성
    season_dummies = pd.get_dummies(features_df['계절'], prefix='계절')
    features_df = pd.concat([features_df, season_dummies], axis=1)
    
    # 결측치 처리
    numeric_columns = features_df.select_dtypes(include=[np.number]).columns
    features_df[numeric_columns] = features_df[numeric_columns].fillna(0)
    
    # 이상치 처리
    if '당월_매출_금액' in features_df.columns:
        for col in ['7일_이동평균_매출', '30일_이동평균_매출']:
            if col in features_df.columns:
                Q1 = features_df[col].quantile(0.25)
                Q3 = features_df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                features_df[col] = features_df[col].clip(lower_bound, upper_bound)
    
    return features_df

In [48]:
# 5번 셀 - 모델 학습 데이터 준비
# 필요한 라이브러리 추가 임포트
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# 분석 대상 상권 정의
target_areas = ['용산전자상가(용산역)', '노량진역(노량진)', '가산디지털단지']
print(f"분석 대상 상권: {target_areas}")

# 날씨 데이터 전처리
print("\n날씨 데이터 전처리 중...")
weather_2022_processed = preprocess_weather_data(weather_2022)
weather_2023_processed = preprocess_weather_data(weather_2023)

# 매출 데이터 전처리
print("\n매출 데이터 전처리 중...")
sales_2022_filtered = sales_2022[sales_2022['상권_코드_명'].isin(target_areas)].copy()
sales_2022_filtered['날짜'] = pd.to_datetime(sales_2022_filtered['기준_년분기_코드'].astype(str).apply(
    lambda x: f"{x[:4]}-{int(x[4:])*3-2}-01"
))
sales_2022_processed = create_sales_features(sales_2022_filtered)

# 날씨 특성 생성
print("\n날씨 특성 생성 중...")
features_2022 = create_weather_features(weather_2022_processed)
features_2023 = create_weather_features(weather_2023_processed)

# 학습 데이터 준비
print("\n데이터 병합 중...")
train_data = pd.merge(features_2022, sales_2022_processed, on='날짜', how='inner')

# 실제 컬럼 확인
print("\n현재 사용 가능한 컬럼:")
print(train_data.columns.tolist())

# 특성과 타겟 분리
feature_columns = [
    # 기상 관련 기본 특성
    '평균기온', '최저기온', '최고기온', '강수량', '평균습도', '평균풍속', '평균기압',
    # 날씨 이동평균 특성
    '평균기온_7일이동평균', '강수량_7일이동평균', '평균습도_7일이동평균',
    '평균기온_30일이동평균', '강수량_30일이동평균', '평균습도_30일이동평균',
    # 날씨 변화 특성
    '기온변화', '전일대비기온변화', '폭염일', '한파일', '강수일', 
    # 시간 관련 특성
    '월', '요일', '주말',
    # 매출 관련 특성
    '7일_이동평균_매출', '30일_이동평균_매출',
    '전일대비_매출_변화', '7일_매출_변화율'
]

# 실제로 존재하는 특성만 선택
available_features = [col for col in feature_columns if col in train_data.columns]
print("\n사용 가능한 특성:", available_features)

print("\n특성 분리 중...")
X = train_data[available_features]
y = train_data['당월_매출_금액']

# 데이터 분할 전 상권별 층화 추출을 위한 준비
stratify = train_data['상권_코드_명']

# 데이터 분할 (상권 정보를 기준으로 층화 추출)
print("데이터 분할 중...")
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=stratify
)

# 특성 스케일링
print("특성 스케일링 중...")
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 타겟값 스케일링
target_scaler = MinMaxScaler(feature_range=(0, 1))
y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_val_scaled = target_scaler.transform(y_val.values.reshape(-1, 1))

# RandomForest 하이퍼파라미터 튜닝
print("하이퍼파라미터 튜닝 중...")
param_grid = {
    'n_estimators': [200],
    'max_depth': [15, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [2, 4],
    'max_features': ['sqrt'],
    'bootstrap': [True]
}

rf_model = RandomForestRegressor(
    random_state=42,
    n_jobs=-1,
    criterion='squared_error',
    oob_score=True
)

grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    scoring='neg_root_mean_squared_error',
    verbose=1
)

# 모델 학습
print("\n모델 학습 중...")
grid_search.fit(X_train_scaled, y_train_scaled.ravel())

print("\n최적 파라미터:", grid_search.best_params_)

# 최적화된 모델로 성능 평가
print("\n모델 성능 평가 중...")
best_model = grid_search.best_estimator_

# 스케일된 데이터에 대한 예측
y_train_pred_scaled = best_model.predict(X_train_scaled)
y_val_pred_scaled = best_model.predict(X_val_scaled)

# 원래 스케일로 역변환
y_train_pred = target_scaler.inverse_transform(y_train_pred_scaled.reshape(-1, 1))
y_val_pred = target_scaler.inverse_transform(y_val_pred_scaled.reshape(-1, 1))

# 성능 지표 계산
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)

print(f"\n모델 성능:")
print(f"Train R² 점수: {train_r2:.4f}")
print(f"Validation R² 점수: {val_r2:.4f}")
print(f"Train RMSE: {train_rmse:,.0f}")
print(f"Validation RMSE: {val_rmse:,.0f}")
print(f"Train MAE: {train_mae:,.0f}")
print(f"Validation MAE: {val_mae:,.0f}")

# 특성 중요도 분석
print("\n특성 중요도 분석 중...")
feature_importance = pd.DataFrame({
    '특성': available_features,
    '중요도': best_model.feature_importances_
}).sort_values('중요도', ascending=False)

print("\n상위 10개 중요 특성:")
print(feature_importance.head(10))

# 전역 변수로 저장
globals().update({
    'feature_scaler': scaler,
    'target_scaler': target_scaler,
    'best_model': best_model,
    'available_features': available_features,
    'weather_2022_processed': weather_2022_processed,
    'weather_2023_processed': weather_2023_processed,
    'features_2022': features_2022,
    'features_2023': features_2023,
    'sales_2022_processed': sales_2022_processed,
    'target_areas': target_areas
})

분석 대상 상권: ['용산전자상가(용산역)', '노량진역(노량진)', '가산디지털단지']

날씨 데이터 전처리 중...

매출 데이터 전처리 중...

날씨 특성 생성 중...

데이터 병합 중...

현재 사용 가능한 컬럼:
['날짜', '평균기온', '최저기온', '최고기온', '강수량', '평균습도', '평균풍속', '평균기압', '평균기온_7일이동평균', '평균기온_30일이동평균', '강수량_7일이동평균', '강수량_30일이동평균', '평균습도_7일이동평균', '평균습도_30일이동평균', '기온변화', '전일대비기온변화', '체감온도', '폭염일', '한파일', '강수일', '강풍일', '월_x', '요일_x', '주말_x', '공휴일', '기준_년분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드', '서비스_업종_코드_명', '당월_매출_금액', '당월_매출_건수', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액', '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수', '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수', '시간대_건수~06_매출_건수', '시간대_건수~11_매출_건수', '시간대_건수~14_매출_건수',

In [50]:
# 6번 셀 - 최종 모델 학습
print("\n최종 모델 학습 중...")

# 최적 파라미터로 모델 생성
final_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=grid_search.best_params_['max_depth'],
    min_samples_split=grid_search.best_params_['min_samples_split'],
    min_samples_leaf=grid_search.best_params_['min_samples_leaf'],
    max_features='sqrt',
    bootstrap=True,
    random_state=42,
    n_jobs=-1,
    criterion='squared_error',
    oob_score=True
)

# 전체 학습 데이터로 모델 학습
X_full_scaled = scaler.fit_transform(X)
y_full_scaled = target_scaler.fit_transform(y.values.reshape(-1, 1))

final_model.fit(X_full_scaled, y_full_scaled.ravel())

# 모델 성능 평가
print("\n최종 모델 성능:")
# Out-of-bag 점수
print(f"Out-of-bag 점수: {final_model.oob_score_:.4f}")

# 학습 데이터에 대한 예측 및 성능 평가
y_pred_scaled = final_model.predict(X_full_scaled)
y_pred = target_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1))

final_r2 = r2_score(y, y_pred)
final_rmse = np.sqrt(mean_squared_error(y, y_pred))
final_mae = mean_absolute_error(y, y_pred)

print(f"전체 데이터 R² 점수: {final_r2:.4f}")
print(f"전체 데이터 RMSE: {final_rmse:,.0f}")
print(f"전체 데이터 MAE: {final_mae:,.0f}")

# 예측값 범위 확인
print("\n예측값 범위 검증:")
print(f"실제 최소값: {y.min():,.0f}")
print(f"실제 최대값: {y.max():,.0f}")
print(f"예측 최소값: {y_pred.min():,.0f}")
print(f"예측 최대값: {y_pred.max():,.0f}")

# 상권별 성능 평가
print("\n상권별 성능 평가:")
for area in train_data['상권_코드_명'].unique():
    area_mask = train_data['상권_코드_명'] == area
    area_y = y[area_mask]
    area_pred = y_pred[area_mask]
    
    area_r2 = r2_score(area_y, area_pred)
    area_rmse = np.sqrt(mean_squared_error(area_y, area_pred))
    area_mae = mean_absolute_error(area_y, area_pred)
    
    print(f"\n{area}:")
    print(f"R² 점수: {area_r2:.4f}")
    print(f"RMSE: {area_rmse:,.0f}")
    print(f"MAE: {area_mae:,.0f}")

# 모델과 스케일러를 전역 변수로 저장
globals().update({
    'final_model': final_model,
    'feature_scaler': scaler,
    'target_scaler': target_scaler,
    'feature_columns': feature_columns
})


최종 모델 학습 중...

최종 모델 성능:
Out-of-bag 점수: 0.5071
전체 데이터 R² 점수: 0.8083
전체 데이터 RMSE: 41,709,555,363
전체 데이터 MAE: 11,569,795,545

예측값 범위 검증:
실제 최소값: 4,132,300
실제 최대값: 668,096,277,728
예측 최소값: 51,838,590
예측 최대값: 549,392,529,567

상권별 성능 평가:

노량진역(노량진):
R² 점수: 0.8467
RMSE: 40,115,549,468
MAE: 8,503,464,033

가산디지털단지:
R² 점수: 0.7043
RMSE: 38,438,306,489
MAE: 11,979,482,976

용산전자상가(용산역):
R² 점수: 0.8288
RMSE: 48,396,804,999
MAE: 15,033,461,763


In [52]:
# 7번 셀 - 2023년 매출 예측
print("2023년 매출 예측 중...")

# 계절 판별 함수 정의
def get_season(month):
    if month in [12, 1, 2]:
        return '겨울'
    elif month in [3, 4, 5]:
        return '봄'
    elif month in [6, 7, 8]:
        return '여름'
    else:
        return '가을'

# 2023년 날짜 생성
dates_2023 = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')

# 전체 예측 결과를 저장할 리스트
predictions_list = []

for area in target_areas:
    print(f"\n{area} 예측 중...")
    
    # 2022년 해당 상권 데이터 (매출 패턴 참조용)
    area_data_2022 = sales_2022_processed[sales_2022_processed['상권_코드_명'] == area]
    
    # 해당 상권의 기본 매출 통계
    base_sales_stats = {
        'mean': area_data_2022['당월_매출_금액'].mean(),
        'std': area_data_2022['당월_매출_금액'].std(),
        'min': area_data_2022['당월_매출_금액'].min(),
        'max': area_data_2022['당월_매출_금액'].max()
    }
    
    for date in dates_2023:
        try:
            # 해당 날짜의 날씨 데이터 준비
            weather_data = features_2023[features_2023['날짜'].dt.date == date.date()].copy()
            
            if len(weather_data) == 0:
                print(f"Warning: {date.date()}의 날씨 데이터가 없습니다.")
                continue
            
            # 기본 특성 데이터 준비
            prediction_features = pd.DataFrame()
            
            # 날씨 관련 특성
            for col in [c for c in available_features if c in weather_data.columns]:
                prediction_features[col] = weather_data[col]
            
            # 시간 관련 특성
            prediction_features['월'] = date.month
            prediction_features['요일'] = date.dayofweek
            prediction_features['주말'] = int(date.dayofweek in [5, 6])
            
            # 매출 관련 특성은 2022년 동일 상권, 동일 월의 평균값 사용
            same_month_data = area_data_2022[
                area_data_2022['날짜'].dt.month == date.month
            ]
            
            prediction_features['7일_이동평균_매출'] = same_month_data['7일_이동평균_매출'].mean()
            prediction_features['30일_이동평균_매출'] = same_month_data['30일_이동평균_매출'].mean()
            prediction_features['전일대비_매출_변화'] = 0  # 예측시점에서는 0으로 초기화
            prediction_features['7일_매출_변화율'] = 0  # 예측시점에서는 0으로 초기화
            
            # 누락된 특성 처리
            for col in available_features:
                if col not in prediction_features.columns:
                    if col in same_month_data.columns:
                        prediction_features[col] = same_month_data[col].mean()
                    else:
                        prediction_features[col] = 0
            
            # 특성 순서 맞추기
            prediction_features = prediction_features[available_features]
            
            # 예측 수행
            X_predict_scaled = feature_scaler.transform(prediction_features)
            prediction_scaled = best_model.predict(X_predict_scaled)
            prediction = target_scaler.inverse_transform(prediction_scaled.reshape(-1, 1))[0][0]
            
            # 예측값 범위 검증 및 조정
            prediction = np.clip(
                prediction,
                base_sales_stats['min'] * 0.7,  # 최소값의 70%
                base_sales_stats['max'] * 1.3   # 최대값의 130%
            )
            
            # 예측 결과 저장
            prediction_data = {
                '날짜': date.strftime('%Y-%m-%d'),
                '상권_코드_명': area,
                '예측_매출': float(prediction),
                '기준_년월': date.strftime('%Y%m'),
                '기준_년분기_코드': f"{date.year}{(date.month-1)//3 + 1}",
                'year': date.year,
                'month': date.month,
                'day': date.day,
                'dayofweek': date.dayofweek
            }
            predictions_list.append(prediction_data)
            
        except Exception as e:
            print(f"Error processing {date} for {area}: {str(e)}")
            continue
    
    print(f"{area} 예측 완료")

print("\n전체 예측 완료")
print(f"생성된 예측 데이터 수: {len(predictions_list)}")

# 예측 결과 검증
if predictions_list:
    predictions_df = pd.DataFrame(predictions_list)
    print("\n예측 결과 기본 통계:")
    print(predictions_df.groupby('상권_코드_명')['예측_매출'].describe())
    
    # 상권별 월별 평균 예측 매출
    print("\n상권별 월별 평균 예측 매출:")
    monthly_pred = predictions_df.groupby(['상권_코드_명', 'month'])['예측_매출'].mean().unstack()
    print(monthly_pred)
    
    # 전역 변수로 저장
    globals().update({
        'predictions_df': predictions_df,
        'monthly_pred': monthly_pred
    })
else:
    print("Warning: 예측 결과가 없습니다!")

2023년 매출 예측 중...

용산전자상가(용산역) 예측 중...
용산전자상가(용산역) 예측 완료

노량진역(노량진) 예측 중...
노량진역(노량진) 예측 완료

가산디지털단지 예측 중...
가산디지털단지 예측 완료

전체 예측 완료
생성된 예측 데이터 수: 1095

예측 결과 기본 통계:
             count          mean           std           min           25%  \
상권_코드_명                                                                      
가산디지털단지      365.0  1.820329e+09  1.056879e+09  9.524423e+08  1.110521e+09   
노량진역(노량진)    365.0  1.355460e+09  6.241542e+08  9.524423e+08  1.104233e+09   
용산전자상가(용산역)  365.0  1.822781e+09  1.338352e+09  9.524423e+08  1.110521e+09   

                      50%           75%           max  
상권_코드_명                                                
가산디지털단지      1.209200e+09  2.565160e+09  5.608245e+09  
노량진역(노량진)    1.189789e+09  1.298679e+09  4.894720e+09  
용산전자상가(용산역)  1.209200e+09  2.161427e+09  8.827691e+09  

상권별 월별 평균 예측 매출:
month                  1             2             3             4   \
상권_코드_명                                                               
가산디지

In [53]:
# 7.5번 셀 - 예측 결과 저장
print("\n예측 결과 저장 중...")

# 예측 결과를 DataFrame으로 변환
predictions_df = pd.DataFrame(predictions_list)

# 기본 통계 확인
print("\n예측 결과 기본 통계:")
print("\n상권별 통계:")
print(predictions_df.groupby('상권_코드_명')['예측_매출'].describe())

print("\n월별 통계:")
print(predictions_df.groupby('month')['예측_매출'].describe())

# 2022년 실제 데이터와 비교
print("\n2022년 실제 데이터와 비교:")
for area in target_areas:
    area_2022 = sales_2022[sales_2022['상권_코드_명'] == area]['당월_매출_금액']
    area_2023 = predictions_df[predictions_df['상권_코드_명'] == area]['예측_매출']
    
    print(f"\n{area}:")
    print(f"2022 평균 매출: {area_2022.mean():,.0f}")
    print(f"2023 예측 평균 매출: {area_2023.mean():,.0f}")
    print(f"변화율: {((area_2023.mean() / area_2022.mean()) - 1) * 100:.1f}%")

# MongoDB에 저장
db_manager = DatabaseManager()
try:
    # 데이터 검증
    print("\n데이터 검증 중...")
    required_columns = ['날짜', '상권_코드_명', '예측_매출', '기준_년월', '기준_년분기_코드']
    missing_columns = [col for col in required_columns if col not in predictions_df.columns]
    
    if missing_columns:
        raise ValueError(f"필수 컬럼 누락: {missing_columns}")
    
    # 데이터 타입 변환
    predictions_df['날짜'] = pd.to_datetime(predictions_df['날짜']).dt.strftime('%Y-%m-%d')
    predictions_df['예측_매출'] = predictions_df['예측_매출'].astype(float)
    
    # 예측 결과 저장
    predictions_list = predictions_df.to_dict('records')
    
    # 기존 데이터 삭제
    print("기존 예측 데이터 삭제 중...")
    collections = db_manager.db.list_collection_names()
    if 'predicted_sales_2023' in collections:
        db_manager.db['predicted_sales_2023'].drop()
    
    # 새로운 예측 결과 저장
    db_manager.save_data(predictions_list, 'predicted_sales_2023')
    print("예측 결과가 MongoDB에 저장되었습니다.")
    
    # 모델 성능 지표 및 중요 정보 저장
    model_info = {
        'model_performance': {
            'train_r2': float(train_r2),
            'val_r2': float(val_r2),
            'train_rmse': float(train_rmse),
            'val_rmse': float(val_rmse),
            'train_mae': float(train_mae),
            'val_mae': float(val_mae)
        },
        'feature_importance': feature_importance.to_dict('records'),
        'prediction_stats': {
            'mean_predicted_sales': float(predictions_df['예측_매출'].mean()),
            'min_predicted_sales': float(predictions_df['예측_매출'].min()),
            'max_predicted_sales': float(predictions_df['예측_매출'].max())
        },
        'area_stats': predictions_df.groupby('상권_코드_명')['예측_매출'].describe().to_dict()
    }
    
    db_manager.save_data([model_info], 'model_info')
    print("모델 정보가 MongoDB에 저장되었습니다.")
    
    # 저장된 데이터 확인
    saved_predictions = pd.DataFrame(db_manager.get_data('predicted_sales_2023'))
    print("\n저장된 데이터 샘플:")
    print(saved_predictions.head())
    print("\n저장된 데이터 크기:", len(saved_predictions))
    
    # 데이터 정합성 검증
    print("\n데이터 정합성 검증:")
    print(f"원본 데이터 건수: {len(predictions_df)}")
    print(f"저장된 데이터 건수: {len(saved_predictions)}")
    
    if len(predictions_df) != len(saved_predictions):
        print("Warning: 데이터 건수가 일치하지 않습니다!")
    
except Exception as e:
    print(f"저장 중 오류 발생: {str(e)}")
finally:
    if db_manager.client:
        db_manager.client.close()
        print("데이터베이스 연결이 종료되었습니다.")


예측 결과 저장 중...

예측 결과 기본 통계:

상권별 통계:
             count          mean           std           min           25%  \
상권_코드_명                                                                      
가산디지털단지      365.0  1.820329e+09  1.056879e+09  9.524423e+08  1.110521e+09   
노량진역(노량진)    365.0  1.355460e+09  6.241542e+08  9.524423e+08  1.104233e+09   
용산전자상가(용산역)  365.0  1.822781e+09  1.338352e+09  9.524423e+08  1.110521e+09   

                      50%           75%           max  
상권_코드_명                                                
가산디지털단지      1.209200e+09  2.565160e+09  5.608245e+09  
노량진역(노량진)    1.189789e+09  1.298679e+09  4.894720e+09  
용산전자상가(용산역)  1.209200e+09  2.161427e+09  8.827691e+09  

월별 통계:
       count          mean           std           min           25%  \
month                                                                  
1       93.0  2.096844e+09  6.035861e+08  9.770618e+08  1.410726e+09   
2       84.0  1.137847e+09  8.055633e+07  9.524423e+08  1.106954e+0

In [54]:
# 8번 셀 - MongoDB 데이터 확인
print("\nMongoDB 데이터 최종 확인 중...")
db_manager = DatabaseManager()

try:
    # 컬렉션 존재 여부 확인
    collections = db_manager.db.list_collection_names()
    print("\n현재 존재하는 컬렉션:", collections)
    
    # predicted_sales_2023 컬렉션 확인
    if 'predicted_sales_2023' in collections:
        predicted_data = pd.DataFrame(db_manager.get_data('predicted_sales_2023'))
        print("\n예측 데이터 기본 정보:")
        print(predicted_data.info())
        
        print("\n예측 데이터 샘플:")
        print(predicted_data.head())
        
        print("\n상권별 데이터 수:")
        print(predicted_data['상권_코드_명'].value_counts())
        
        print("\n상권별 예측 매출 통계:")
        print(predicted_data.groupby('상권_코드_명')['예측_매출'].describe())
        
        # 날짜 범위 확인
        predicted_data['날짜'] = pd.to_datetime(predicted_data['날짜'])
        print("\n예측 데이터 날짜 범위:")
        print(f"시작일: {predicted_data['날짜'].min()}")
        print(f"종료일: {predicted_data['날짜'].max()}")
        
        # 데이터 연속성 확인
        date_range = pd.date_range(
            start=predicted_data['날짜'].min(),
            end=predicted_data['날짜'].max(),
            freq='D'
        )
        missing_dates = set(date_range) - set(predicted_data['날짜'])
        if missing_dates:
            print("\nWarning: 누락된 날짜가 있습니다:")
            print(sorted(missing_dates))
    else:
        print("\nWarning: predicted_sales_2023 컬렉션이 존재하지 않습니다.")
    
    # model_info 컬렉션 확인
    if 'model_info' in collections:
        model_info = pd.DataFrame(db_manager.get_data('model_info'))
        print("\n모델 정보:")
        if not model_info.empty:
            print(model_info.iloc[0]['model_performance'])
            print("\n상위 5개 중요 특성:")
            feature_importance = pd.DataFrame(model_info.iloc[0]['feature_importance'])
            print(feature_importance.head())
        else:
            print("Warning: model_info 컬렉션이 비어있습니다.")
    else:
        print("\nWarning: model_info 컬렉션이 존재하지 않습니다.")

except Exception as e:
    print(f"\nError: {str(e)}")
finally:
    if db_manager.client:
        db_manager.client.close()
        print("\n데이터베이스 연결이 종료되었습니다.")

print("\n전처리 및 예측 프로세스 완료!")

# 메모리 정리
import gc
gc.collect()


MongoDB 데이터 최종 확인 중...

현재 존재하는 컬렉션: ['sales_2022', 'predicted_sales_2023', 'weather_2022', 'weather_2023', 'model_info']

예측 데이터 기본 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   날짜         1095 non-null   object 
 1   상권_코드_명    1095 non-null   object 
 2   예측_매출      1095 non-null   float64
 3   기준_년월      1095 non-null   object 
 4   기준_년분기_코드  1095 non-null   object 
 5   year       1095 non-null   int64  
 6   month      1095 non-null   int64  
 7   day        1095 non-null   int64  
 8   dayofweek  1095 non-null   int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 77.1+ KB
None

예측 데이터 샘플:
           날짜      상권_코드_명         예측_매출   기준_년월 기준_년분기_코드  year  month  day  \
0  2023-01-01  용산전자상가(용산역)  2.214262e+09  202301     20231  2023      1    1   
1  2023-01-02  용산전자상가(용산역)  1.996661e+09  202301     20231  2023      1    2

816